In [ ]:
import numpy as np
from python.py_rle import mask2rle, rle2mask 
from cython_rle import cy_mask_to_rle
from py_multi_backend import arr2rle, c_order_mask2rle, f_order_mask2rle, thread_arr2rle, thread_f_order_mask2rle, thread_c_order_mask2rle
from py_multi_backend import rle2mask as rust_rle2mask

In [ ]:
SIZE = (3072, 3072)
rand_mask = np.random.random(SIZE) < 0.5
static_mask = np.ones(SIZE) < 0.5
rand_rle = arr2rle(rand_mask.ravel("F"))
# rand_rle = arr2rle(static_mask.ravel("F"))

In [ ]:
%%timeit
_ = rust_rle2mask(rand_rle, SIZE)

In [ ]:
rust_rle2mask(rand_rle, SIZE)

In [ ]:
from pycocotools import mask as cocomask
seg_rle = cocomask.encode(np.asfortranarray(static_mask))
seg_rle["counts"] = seg_rle["counts"].decode("utf-8")

In [ ]:
%%timeit
_ = rle2mask(seg_rle)

In [ ]:
%%timeit
_ = arr2rle(rand_mask.ravel("F"))

In [ ]:
%%timeit
_ = arr2rle(static_mask.ravel("F"))

In [ ]:
%%timeit
_ = f_order_mask2rle(rand_mask)

In [ ]:
%%timeit
_ = f_order_mask2rle(static_mask)

In [ ]:
%%timeit
_ = c_order_mask2rle(rand_mask)

In [ ]:
%%timeit
_ = c_order_mask2rle(static_mask)

In [ ]:
%%timeit
_ = cy_mask_to_rle(rand_mask)

In [ ]:
%%timeit
_ = cy_mask_to_rle(static_mask)

In [ ]:
%%timeit
_ = py_mask_to_rle(rand_mask)

In [ ]:
%%timeit
_ = py_mask_to_rle(static_mask)

In [ ]:
assert cy_mask_to_rle(rand_mask)["counts"] == py_mask_to_rle(rand_mask)["counts"]
assert arr2rle(rand_mask.ravel(order="F")) == py_mask_to_rle(rand_mask)["counts"]
assert f_order_mask2rle(rand_mask) == py_mask_to_rle(rand_mask)["counts"]
# assert go_mask_to_rle(rand_mask.ravel(order="F")) == py_mask_to_rle(rand_mask)["counts"]